In [ ]:
import os
import numpy as np
import h5py

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors


import scipy as scp

import skimage.exposure
from skimage.io import imread
from skimage.filters import *
from skimage.morphology import *
from skimage import data
from skimage import img_as_float
from skimage.morphology import disk
from skimage import measure

plt.close("all")

# Image coordinates are
# top-left corner is (0,0)  and (10,4) addresses pixel on the 10th row and 4th colum

# experiment 5
# start is at 338
r = range(609,615)
expnr = 5
pxPerMeter = 967.5931
dt=0.001;
offset = [783,286]

background = img_as_float(imread('testData/exp%i-0001.jpg' % (expnr), as_grey = True))
treemask = img_as_float(imread('testData/exp%i-treemask-0001.jpg' % (expnr), as_grey = True))

height  = background.shape[0];
width = background.shape[1];
mask = np.ones(background.shape,np.uint8)

# bounds (with x-Axis to the right, y-Axis to the left, like ACCIV)
bounds = np.array([-offset[0],width-offset[0],
                   -(height-offset[1]), offset[1] ]) / pxPerMeter


count = 0
lastBinary = None
for i in r:
    
    frame = 'testData/exp%i-%04i.jpg' % (expnr,i)
    print("Process Frame: ", frame)
    
    image =  imread(frame, as_grey = True)   
    #images.append(image)
    print("Index (0,0) (top-left):", image[0,0])
    print("Index (1,0):", image[1,0])
    
    # testing image processing stuff =================================================
    
    image_clahe = skimage.exposure.equalize_adapthist(image,ntiles_x=16,ntiles_y=16);
    
    
    print("process image")
    
    # background subtract 
    subtracted = np.abs(image - background)
    
    th = threshold_otsu(subtracted)
    # threshol with otsu
    binary = (subtracted > th).astype(np.uint8)

    d = disk(5)
    # do some opening
    binary2 = binary_opening(binary, d)
    # do some thresholding value*otsu
    binary3 = subtracted > 0.3*th
    
    
    
    # remove small objects
    denoise = remove_small_objects(binary3,min_size=5000) # inplace = True
    # thicken up white area abit
    dilated = binary_dilation(denoise, disk(3))
    # blur the image
    blurred = gaussian_filter(dilated,sigma=4) # inplace = True
    
    masked = denoise * image
    
    # find max. length contour (interpolate)
    contours = measure.find_contours(blurred, 0.5)
    if contours :
        contourMax = max(contours,key=lambda x: x.shape[0])
        tck,u = scp.interpolate.splprep([contourMax[:, 0], contourMax[:, 1]], s=1e4)
        unew = np.linspace(0, 1.0, 3000)
        contourInterp = scp.interpolate.splev(unew, tck)
    else:
        contourMax = None
        contourInterp = None
        
    
    final_mask = treemask * binary3.astype(np.uint8)
    # =================================================================================
    
    
    # Save some images ================================================================
    skimage.io.imsave('testData/exp%i-clahe-%04i.jpg' % (expnr,i), image_clahe )
    skimage.io.imsave('testData/exp%i-masked-%04i.jpg' % (expnr,i), image*final_mask )
    skimage.io.imsave('testData/exp%i-subtracted-%04i.jpg' % (expnr,i), subtracted )
    # =================================================================================
    
    # Plotting ========================================================================
    
    fig, axes = plt.subplots(ncols=3,nrows=3, figsize=(15, 8), sharex=True, sharey=True)
    fig.set_tight_layout(True)
    i = 1
    for row in axes:
        for a in row:
            exec("ax%i=a" % i)
            i+=1
    
    #plt.matshow( image_clahe , cmap=cm.Greys_r, interpolation='none' );
    
    ax1.imshow(image,cmap=cm.Greys_r)
    ax1.set_title(r'original image')
    ax1.axis('off')
    
    ax2.imshow(subtracted,cmap=cm.Greys_r, interpolation='none')
    ax2.set_title(r'image$\rightarrow$subtracted , background subtracted')
    ax2.axis('off')
                  
    
    ax3.imshow(binary,cmap=cm.Greys_r, interpolation='none')
    ax3.set_title(r'subtracted$\rightarrow$binary , otsu threshold')
    ax3.axis('off')
    
    ax4.imshow(binary2,cmap=cm.Greys_r, interpolation='none')
    ax4.set_title(r'binary$\rightarrow$binary2 ,  morph. opening')
    ax4.axis('off')
    
    ax5.imshow(binary3,cmap=cm.Greys_r, interpolation='none')
    ax5.set_title(r'subtracted$\rightarrow$binary3 , value*otsu threshold')
    ax5.axis('off')

    
    ax6.imshow(denoise,cmap=cm.Greys_r, interpolation='none')
    ax6.axis('off')
    ax6.set_title(r'binary3$\rightarrow$denoise, removed small objects')
    
    
    ax7.imshow(dilated,cmap=cm.Greys_r, interpolation='none')
    ax7.axis('off')
    ax7.set_title(r'denoise$\rightarrow$dilated, dilation to thicken up')
    
    ax8.imshow(blurred,cmap=cm.Greys_r, interpolation='none')
    ax8.axis('off')
    ax8.set_title(r'dilated$\rightarrow$blurred, gaussian filter')
    
    
    for n, contour in enumerate(contours):
        ax8.plot(contour[:, 1], contour[:, 0], 'r', linewidth=1)
    #ax3.plot(contourMax[:,1], contourMax[:,0], "bo", linewidth=2 )   
    ax8.plot(contourInterp[1], contourInterp[0], "r", linewidth=2)    
    
    
    ax9.imshow(image,cmap=cm.Greys_r, interpolation='none')

    ax9.imshow(np.ma.masked_where(final_mask==0,final_mask),
               cmap=cm.jet, alpha=0.8 ,interpolation='none')
    ax9.axis('off')
    ax9.set_title('masked')
    # =================================================================================
    
    
    # Make Contour Frames
#     dpi=100
#     f = plt.figure(num=None, frameon=False, figsize=(background.shape[1]/dpi,background.shape[0]/dpi), dpi=dpi)
#     ax = plt.Axes(f, [0., 0., 1., 1.])
#     #ax.set_axis_off()
#     f.add_axes(ax)
#     ax.imshow(image,cmap=cm.Greys_r)
#     ax.set_autoscalex_on(False)
#     ax.set_autoscaley_on(False)
#     if contourInterp:
#           ax.plot(contourInterp[1], contourInterp[0], "b", linewidth=2)
#     f.savefig("Contour%04i.jpg" % count)
#     plt.close(f)
#     f=None
#     ax=None
    
    # Make Mask Frames
#     dpi=100
#     f = plt.figure(num=None, frameon=False, figsize=(background.shape[1]/dpi,background.shape[0]/dpi), dpi=dpi)
#     ax = plt.Axes(f, [0., 0., 1., 1.])
#     #ax.set_axis_off()
#     f.add_axes(ax)
#     ax.imshow(binary3,cmap=cm.Greys_r)
#     ax.set_autoscalex_on(False)
#     ax.set_autoscaley_on(False)
#     f.savefig("Mask%04i.jpg" % count)
#     plt.close(f)
#     f=None
#     ax=None
    
    
    time = count*dt;
    h5File = h5py.File('testData/image%03i.h5'%count, 'w')
    dataset = h5File.create_dataset("bounds", data=np.array(bounds))
    dataset = h5File.create_dataset("data", data=np.flipud(image)) # flip image,mask for ACCIV
    dataset = h5File.create_dataset("mask", data=np.flipud(mask))
    dataset = h5File.create_dataset("velocityMask", data=np.flipud(final_mask))
    dataset = h5File.create_dataset("time", data=np.array(time))
    h5File.close()

    
    count+=1
    
plt.show()

Process Frame:  testData/exp5-0609.jpg
Index (0,0) (top-left): 0.117647058824
Index (1,0): 0.121568627451
process image

/opt/python3.4env/lib/python3.4/site-packages/skimage/util/dtype.py:107: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))
/opt/python3.4env/lib/python3.4/site-packages/skimage/util/dtype.py:107: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))



Process Frame:  testData/exp5-0610.jpg
Index (0,0) (top-left): 0.117647058824
Index (1,0): 0.121568627451
process image
Process Frame:  testData/exp5-0611.jpg
Index (0,0) (top-left): 0.117647058824
Index (1,0): 0.125490196078
process image
Process Frame:  testData/exp5-0612.jpg
Index (0,0) (top-left): 0.113725490196
Index (1,0): 0.121568627451
process image
Process Frame:  testData/exp5-0613.jpg
Index (0,0) (top-left): 0.113725490196
Index (1,0): 0.117647058824
process image
Process Frame:  testData/exp5-0614.jpg
Index (0,0) (top-left): 0.109803921569
Index (1,0): 0.117647058824
process image

In [ ]:
import numpy as np

